# பயண வாடிக்கையாளர் ஆதரவு மற்றும் ஹேண்ட்ஆஃப் ஒர்கெஸ்ட்ரேஷன்

இந்த நோட்புக் Microsoft Agent Framework-ஐப் பயன்படுத்தி **ஹேண்ட்ஆஃப் ஒர்கெஸ்ட்ரேஷன்**-ஐ விளக்குகிறது. வாடிக்கையாளர் தேவைகளின் அடிப்படையில் முகவர்கள் நிபுணர்களுக்கு கட்டுப்பாட்டை மாற்றும் பயண வாடிக்கையாளர் ஆதரவு அமைப்பை உருவாக்குவோம்.

## நீங்கள் கற்றுக்கொள்ளப்போகிறீர்கள்:
1. **ஹேண்ட்ஆஃப் ஒர்கெஸ்ட்ரேஷன்**: சூழல் மற்றும் நிபுணத்துவத்தின் அடிப்படையில் டைனமிக் ஏஜென்ட் ரவுடிங்
2. **HandoffBuilder**: ஹேண்ட்ஆஃப் வேலைப்பாடுகளை உருவாக்குவதற்கான உயர் நிலை API
3. **நிபுணர் ரவுடிங்**: முகவர்கள் மற்ற முகவர்களுக்கு டைனமிக்காக ஹேண்ட்ஆஃப் செய்ய முடியும்
4. **பல முறை உரையாடல்கள்**: ஹேண்ட்ஆஃப்-களுக்கு இடையே தொடர்ச்சியான சூழல் பாதுகாப்பு
5. **வாடிக்கையாளர் ஆதரவு ஓட்டம்**: முகவர் ஹேண்ட்ஆஃப்-களின் உண்மையான உலக பயன்பாடு


## முன்பதிவுகள்:
- Microsoft Agent Framework நிறுவப்பட்டுள்ளது
- GitHub token அல்லது OpenAI API key அமைக்கப்பட்டுள்ளது
- அடிப்படை முகவர் கருத்துக்களைப் புரிந்துகொள்வது


In [5]:
import asyncio
import json
import os
from collections.abc import AsyncIterable
from typing import Any, cast

from agent_framework import (
    ChatMessage,
    HandoffBuilder,
    HandoffUserInputRequest,
    RequestInfoEvent,
    WorkflowEvent,
    WorkflowOutputEvent,
    WorkflowRunState,
    WorkflowStatusEvent,
)

# GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

ImportError: cannot import name 'HandoffBuilder' from 'agent_framework' (/Users/koreypace/ai-agents-for-beginners/.venv/lib/python3.12/site-packages/agent_framework/__init__.py)

## படி 1: கட்டமைக்கப்பட்ட வெளியீடுகளுக்கான Pydantic மாதிரிகளை வரையறுக்கவும்

இந்த மாதிரிகள் ஒவ்வொரு சிறப்பு முகவரும் திருப்பும் சிக்கலான அமைப்பை வரையறுக்கின்றன. இது அனைத்து முகவர்களிடமிருந்தும் ஒரே மாதிரியான மற்றும் பகுப்பாய்வு செய்யக்கூடிய பதில்களை உறுதிசெய்கிறது.


In [2]:
class FlightBookingResult(BaseModel):
    """Flight booking confirmation from the booking agent."""

    destination: str
    departure_date: str
    return_date: str
    booking_reference: str
    passenger_name: str
    flight_details: str
    total_cost: str
    status: str


class DisputeResult(BaseModel):
    """Dispute resolution result from the disputes agent."""

    dispute_type: str
    original_booking: str
    refund_amount: str
    refund_method: str
    processing_time: str
    reference_number: str
    status: str


class TripCheckResult(BaseModel):
    """Trip confirmation result from the trip check agent."""

    trip_reference: str
    destination: str
    travel_dates: str
    confirmation_status: str
    special_notes: str
    contact_info: str

NameError: name 'BaseModel' is not defined

## படி 2: சூழல் மாறிகள் ஏற்றவும்


In [ ]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"),
    api_key=os.environ.get("GITHUB_TOKEN"),
    model_id="gpt-4o"
)

## படி 3: நான்கு சிறப்பு பயண ஆதரவு முகவர்களை உருவாக்கவும்

ஒவ்வொரு முகவருக்கும் குறிப்பிட்ட நிபுணத்துவம் உள்ளது மற்றும் வாடிக்கையாளர் தேவைகளின் அடிப்படையில் பொருத்தமான நிபுணர்களுக்கு ஒப்படைக்க முடியும்.


In [ ]:
# Agent 1: Customer Support Agent (Main triage agent)
customer_support_agent = chat_client.create_agent(
    instructions=(
        "You are a friendly customer support agent for a travel company. "
        "Assess customer requests and route them to the appropriate specialist: "
        "- For flight bookings or reservations: call handoff_to_booking_agent "
        "- For refunds, disputes, or billing issues: call handoff_to_disputes_agent "
        "- For trip confirmations or travel plan checks: call handoff_to_trip_check_agent "
        "Be welcoming and ensure customers feel heard before routing them."
    ),
    name="customer_support_agent",
)


# Agent 2: Booking Agent (Flight booking specialist)
booking_agent = chat_client.create_agent(
    instructions=(
        "You are a flight booking specialist. Handle all flight reservations and bookings. "
        "When a customer wants to book a flight, collect their destination, travel dates, "
        "and confirm the booking. Always provide a booking reference number. "
        "Return structured JSON with booking details. "
        "The flight is always confirmed as booked regardless of destination."
    ),
    name="booking_agent",
    response_format=FlightBookingResult,
)

# Agent 3: Disputes Agent (Refund and billing specialist)
disputes_agent = chat_client.create_agent(
    instructions=(
        "You are a disputes and refunds specialist. Handle customer complaints, "
        "refund requests, and billing disputes. Always approve refunds and provide "
        "a reference number. Process refunds back to the original payment method. "
        "Return structured JSON with refund details. "
        "All refund requests are approved and processed immediately."
    ),
    name="disputes_agent",
    response_format=DisputeResult,
)
# Agent 4: Trip Check Agent (Travel confirmation specialist)
trip_check_agent = chat_client.create_agent(
    instructions=(
        "You are a travel confirmation specialist. Verify and confirm customer "
        "travel plans, check itineraries, and provide travel status updates. "
        "Always confirm that travel plans are in order and provide reassurance. "
        "Return structured JSON with confirmation details. "
        "All travel plans are confirmed as valid and ready."
    ),
    name="trip_check_agent",
    response_format=TripCheckResult,
)




## படி 4: ஹேண்ட்ஆஃப் வேலைப்பாடுகளை உருவாக்கவும்

HandoffBuilder ஒரு வேலைப்பாடுகளை உருவாக்குகிறது, இதில் வாடிக்கையாளர் ஆதரவு முகவர் வாடிக்கையாளர் தேவைகளின் அடிப்படையில் நிபுணர்களுக்கு தன்னிச்சையாக மாற்றம் செய்ய முடியும்.


In [ ]:
workflow = (
    HandoffBuilder(
        name="travel_support_handoff",
        participants=[customer_support_agent, booking_agent, disputes_agent, trip_check_agent],
    )
    .set_coordinator(customer_support_agent)  # Main agent that receives initial requests
    .add_handoff(customer_support_agent, [booking_agent, disputes_agent, trip_check_agent])
    .with_termination_condition(
        lambda conv: sum(1 for msg in conv if msg.role.value == "user") > 3
    )  # Stop after 3 user messages
    .build()
)

display(HTML("""
<div style='padding: 20px; background: linear-gradient(135deg, #ff7043 0%, #ff5722 100%); color: white; border-radius: 8px; margin: 10px 0;'>
    <h3 style='margin: 0 0 15px 0;'>Handoff Workflow Built Successfully!</h3>
    <p style='margin: 0; line-height: 1.6;'>
        <strong>Handoff Flow:</strong><br>
        • User Request → <strong>Customer Support Agent</strong> (triage)<br>
        • Support Agent → <strong>Specialist Agent</strong> (dynamic handoff)<br>
        • Specialist → <strong>Resolution</strong> (expert handling)<br>
        • System → <strong>User Response</strong> (final result)
    </p>
</div>
"""))



## படி 5: நிகழ்வுகளை செயல்படுத்த உதவியாளர்கள்

இந்த செயல்பாடுகள் வேலைநிலை நிகழ்வுகளை செயல்படுத்தவும், கையளிப்பு செயல்முறையின் போது பயனர் உள்ளீட்டு கோரிக்கைகளை நிர்வகிக்கவும் உதவுகின்றன.


In [ ]:
async def drain_events(stream: AsyncIterable[WorkflowEvent]) -> list[WorkflowEvent]:
    """Collect all events from an async stream into a list."""
    return [event async for event in stream]


def handle_workflow_events(events: list[WorkflowEvent]) -> list[RequestInfoEvent]:
    """Process workflow events and extract pending user input requests."""
    requests: list[RequestInfoEvent] = []
    
    for event in events:
        if isinstance(event, WorkflowStatusEvent) and event.state in {
            WorkflowRunState.IDLE,
            WorkflowRunState.IDLE_WITH_PENDING_REQUESTS,
        }:
           print(f"[Workflow Status] {event.state.name}")

        elif isinstance(event, WorkflowOutputEvent):
            conversation = cast(list[ChatMessage], event.data)
            if isinstance(conversation, list):
                print("\n=== Final Conversation ===")
                for message in conversation:
                    # Filter out messages with no text (tool calls)
                    if not message.text.strip():
                        continue
                    speaker = message.author_name or message.role.value
                    print(f"- {speaker}: {message.text}")
                print("==========================")

        elif isinstance(event, RequestInfoEvent):
            if isinstance(event.data, HandoffUserInputRequest):
                print_handoff_request(event.data)
                requests.append(event)

    return requests


def print_handoff_request(request: HandoffUserInputRequest) -> None:
    """Display a user input request with conversation context."""
    print("\n=== User Input Requested ===")
    # Filter out messages with no text for cleaner display
    messages_with_text = [
        msg for msg in request.conversation if msg.text.strip()]
    print(f"Last {len(messages_with_text)} messages in conversation:")
    for message in messages_with_text[-3:]:  # Show last 3 for brevity
        speaker = message.author_name or message.role.value
        text = message.text[:100] + \
            "..." if len(message.text) > 100 else message.text
        print(f"  {speaker}: {text}")
    print("============================")


print("Helper functions defined for event processing")

## படி 6: சோதனை வழக்கு 1 - விமான முன்பதிவு கோரிக்கை

விமான முன்பதிவு கோரிக்கையுடன் எங்கள் ஒப்படை வேலைநடத்தை சோதிப்போம். வாடிக்கையாளர் ஆதரவு முகவர் முன்பதிவு முகவருக்கு ஒப்படைக்க வேண்டும்.


In [ ]:
async def test_booking_handoff():
    """Test handoff workflow for flight booking requests."""

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Test Case 1: Flight Booking Request</h3>
        <p style='margin: 0;'><strong>Expected Flow:</strong> Customer Support → Booking Agent</p>
    </div>
    """))

    # Start the workflow
    print("[User]: I want to book a flight to Paris for next month")
    events = await drain_events(
        workflow.run_stream("I want to book a flight to Paris for next month")
    )
    pending_requests = handle_workflow_events(events)

    # Handle any additional user input requests
    scripted_responses = [
        "I'd like to travel from New York to Paris on December 15th and return on December 22nd.",
        "Yes, please confirm the booking under the name John Smith."
    ]

    response_index = 0
    while pending_requests and response_index < len(scripted_responses):
        user_response = scripted_responses[response_index]
        print(f"\n[User]: {user_response}")

        responses = {req.request_id: user_response for req in pending_requests}
        events = await drain_events(workflow.send_responses_streaming(responses))
        pending_requests = handle_workflow_events(events)

        response_index += 1

    # Extract and display the final booking result
    if events:
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "booking_agent" and message.text.strip():
                        try:
                            booking_data = FlightBookingResult.model_validate_json(
                                message.text)
                            display_booking_result(booking_data)
                        except Exception as e:
                            print(f"Could not parse booking result: {e}")


def display_booking_result(booking: FlightBookingResult):
    """Display flight booking result in a formatted section."""

    display(HTML(f"""
    <div style='padding: 20px; background: #e8f5e9; border-radius: 8px; margin: 15px 0; border-left: 4px solid #4caf50;'>
        <h3 style='margin: 0 0 15px 0; color: #2e7d32;'>✈️ Flight Booking Confirmed</h3>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
            <div>
                <strong style='color: #333;'>Booking Reference:</strong> {booking.booking_reference}<br>
                <strong style='color: #333;'>Passenger:</strong> {booking.passenger_name}<br>
                <strong style='color: #333;'>Status:</strong> <span style='color: #4caf50; font-weight: bold;'>{booking.status}</span>
            </div>
            <div>
                <strong style='color: #333;'>Destination:</strong> {booking.destination}<br>
                <strong style='color: #333;'>Total Cost:</strong> {booking.total_cost}<br>
                <strong style='color: #333;'>Departure:</strong> {booking.departure_date}
            </div>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Flight Details:</strong> {booking.flight_details}
        </div>
        <div style='background: rgba(76,175,80,0.1); padding: 10px; border-radius: 4px; margin-top: 10px;'>
            <strong style='color: #2e7d32;'>✅ Success:</strong> Flight booking completed through handoff to booking specialist
        </div>
    </div>
    """))


# Run the booking test
await test_booking_handoff()

## படி 7: சோதனை வழக்கு 2 - மோதல்/பணம் திரும்ப கோரிக்கை

பணம் திரும்ப கோரிக்கையுடன் எங்கள் ஒப்படை வேலைநடத்தை சோதிப்போம். வாடிக்கையாளர் ஆதரவு முகவர் மோதல் முகவருக்கு ஒப்படைக்க வேண்டும்.


In [ ]:
async def test_dispute_handoff():
    """Test handoff workflow for dispute/refund requests."""

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Test Case 2: Refund Request</h3>
        <p style='margin: 0;'><strong>Expected Flow:</strong> Customer Support → Disputes Agent</p>
    </div>
    """))

    # Start the workflow
    print("[User]: I need to cancel my flight and get a refund")
    events = await drain_events(
        workflow.run_stream("I need to cancel my flight and get a refund")
    )
    pending_requests = handle_workflow_events(events)

    # Handle any additional user input requests
    scripted_responses = [
        "My booking reference is FL12345. I can't travel due to a family emergency.",
        "Yes, please process the full refund back to my credit card."
    ]

    response_index = 0
    while pending_requests and response_index < len(scripted_responses):
        user_response = scripted_responses[response_index]
        print(f"\n[User]: {user_response}")

        responses = {req.request_id: user_response for req in pending_requests}
        events = await drain_events(workflow.send_responses_streaming(responses))
        pending_requests = handle_workflow_events(events)

        response_index += 1

    # Extract and display the final dispute result
    if events:
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "disputes_agent" and message.text.strip():
                        try:
                            dispute_data = DisputeResult.model_validate_json(
                                message.text)
                            display_dispute_result(dispute_data)
                        except Exception as e:
                            print(f"Could not parse dispute result: {e}")


def display_dispute_result(dispute: DisputeResult):
    """Display dispute resolution result in a formatted section."""

    display(HTML(f"""
    <div style='padding: 20px; background: #fff3e0; border-radius: 8px; margin: 15px 0; border-left: 4px solid #ff9800;'>
        <h3 style='margin: 0 0 15px 0; color: #f57c00;'>💰 Refund Processed</h3>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
            <div>
                <strong style='color: #333;'>Reference Number:</strong> {dispute.reference_number}<br>
                <strong style='color: #333;'>Dispute Type:</strong> {dispute.dispute_type}<br>
                <strong style='color: #333;'>Status:</strong> <span style='color: #ff9800; font-weight: bold;'>{dispute.status}</span>
            </div>
            <div>
                <strong style='color: #333;'>Refund Amount:</strong> {dispute.refund_amount}<br>
                <strong style='color: #333;'>Refund Method:</strong> {dispute.refund_method}<br>
                <strong style='color: #333;'>Processing Time:</strong> {dispute.processing_time}
            </div>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Original Booking:</strong> {dispute.original_booking}
        </div>
        <div style='background: rgba(255,152,0,0.1); padding: 10px; border-radius: 4px; margin-top: 10px;'>
            <strong style='color: #f57c00;'>✅ Success:</strong> Refund processed through handoff to disputes specialist
        </div>
    </div>
    """))

    # Run the dispute test
await test_dispute_handoff()

## படி 8: சோதனை வழக்கு 3 - பயண உறுதிப்படுத்தல் கோரிக்கை

நாம் பயண உறுதிப்படுத்தல் கோரிக்கையுடன் எங்கள் ஒப்படை வேலைநடத்தை சோதிக்கலாம். வாடிக்கையாளர் ஆதரவு முகவர் பயண சரிபார்ப்பு முகவருக்கு ஒப்படைக்க வேண்டும்.


In [ ]:
async def test_trip_check_handoff():
    """Test handoff workflow for trip confirmation requests."""

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Test Case 3: Trip Confirmation</h3>
        <p style='margin: 0;'><strong>Expected Flow:</strong> Customer Support → Trip Check Agent</p>
    </div>
    """))

    # Start the workflow
    print("[User]: Can you confirm my travel plans are all set?")
    events = await drain_events(
        workflow.run_stream("Can you confirm my travel plans are all set?")
    )
    pending_requests = handle_workflow_events(events)

    # Handle any additional user input requests
    scripted_responses = [
        "I'm traveling to London next week. My confirmation number is TR98765.",
        "Perfect, thank you for checking everything is ready!"
    ]

    response_index = 0
    while pending_requests and response_index < len(scripted_responses):
        user_response = scripted_responses[response_index]
        print(f"\n[User]: {user_response}")
        
        responses = {req.request_id: user_response for req in pending_requests}
        events = await drain_events(workflow.send_responses_streaming(responses))
        pending_requests = handle_workflow_events(events)
        
        response_index += 1
    # Extract and display the final trip check result
    if events:
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "trip_check_agent" and message.text.strip():
                        try:
                            trip_data = TripCheckResult.model_validate_json(
                                message.text)
                            display_trip_check_result(trip_data)
                        except Exception as e:
                            print(f"Could not parse trip check result: {e}")


def display_trip_check_result(trip: TripCheckResult):
    """Display trip confirmation result in a formatted section."""

    display(HTML(f"""
    <div style='padding: 20px; background: #f3e5f5; border-radius: 8px; margin: 15px 0; border-left: 4px solid #9c27b0;'>
        <h3 style='margin: 0 0 15px 0; color: #7b1fa2;'>🎯 Trip Confirmed</h3>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-bottom: 15px;'>
            <div>
                <strong style='color: #333;'>Trip Reference:</strong> {trip.trip_reference}<br>
                <strong style='color: #333;'>Destination:</strong> {trip.destination}<br>
                <strong style='color: #333;'>Status:</strong> <span style='color: #9c27b0; font-weight: bold;'>{trip.confirmation_status}</span>
            </div>
            <div>
                <strong style='color: #333;'>Travel Dates:</strong> {trip.travel_dates}<br>
                <strong style='color: #333;'>Contact Info:</strong> {trip.contact_info}
            </div>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Special Notes:</strong> {trip.special_notes}
        </div>
        <div style='background: rgba(156,39,176,0.1); padding: 10px; border-radius: 4px; margin-top: 10px;'>
            <strong style='color: #7b1fa2;'>✅ Success:</strong> Trip confirmed through handoff to trip check specialist
        </div>
    </div>
    """))


# Run the trip check test
await test_trip_check_handoff()

   

## படி 9: வேலைநிலை பகுப்பாய்வு - ஒப்படைப்பு ஓட்டத்தை புரிந்துகொள்வது


In [ ]:
async def analyze_handoff_patterns():
    """Analyze different handoff patterns and routing decisions."""

    display(HTML("""
    <div style='padding: 20px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #7b1fa2;'>Handoff Pattern Analysis</h3>
        <p style='margin: 0;'>Testing different request types to show routing decisions...</p>
    </div>
    """))

    test_requests = [
        "I want to book a round-trip flight to Tokyo",
        "I need a refund for my cancelled flight",
        "Please check if my travel itinerary is confirmed",
        "Can you help me with a billing dispute?"
    ]

    for i, request in enumerate(test_requests, 1):
        print(f"\n--- Test Request {i} ---")
        print(f"User: {request}")

        # Run workflow and capture routing decision
        events = await drain_events(workflow.run_stream(request))

        # Analyze which agent was activated
        for event in events:
            if isinstance(event, WorkflowOutputEvent):
                conversation = cast(list[ChatMessage], event.data)
                for message in conversation:
                    if message.author_name == "customer_support_agent":
                        print(f"Support Agent: {message.text[:100]}...")
                    elif message.author_name in ["booking_agent", "disputes_agent", "trip_check_agent"]:
                        agent_type = {
                            "booking_agent": "🛫 BOOKING SPECIALIST",
                            "disputes_agent": "💰 DISPUTES SPECIALIST",
                            "trip_check_agent": "🎯 TRIP CHECK SPECIALIST"
                        }[message.author_name]
                        print(f"Routed to: {agent_type}")
                        break
                break
    display(HTML("""
    <div style='padding: 25px; background: linear-gradient(135deg, #9c27b0 0%, #673ab7 100%); color: white; border-radius: 12px; 
                box-shadow: 0 4px 12px rgba(156,39,176,0.4); margin: 20px 0;'>
        <h2 style='margin: 0 0 20px 0;'>Handoff Analysis Results</h2>
        <div style='background: rgba(255,255,255,0.15); padding: 15px; border-radius: 8px;'>
            <h4 style='margin: 0 0 10px 0;'>Key Observations</h4>
            <ul style='margin: 0; padding-left: 20px; line-height: 1.6;'>
                <li><strong>Dynamic Routing:</strong> Customer support agent analyzes request intent</li>
                <li><strong>Context Preservation:</strong> Full conversation history maintained</li>
                <li><strong>Specialist Focus:</strong> Each agent handles their expertise area</li>
                <li><strong>Seamless Handoff:</strong> Users don't need to repeat information</li>
            </ul>
        </div>
    </div>
    """))

    # Run the analysis
await analyze_handoff_patterns()


---

**புறக்கணிப்பு**:  
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
